In [1]:
# 1. Generate dataset 
# 2. Train the classifier 
# 3. Detect the face and named it if it is already in our dataset 


# Generate Dataset

In [2]:
import cv2
print(cv2.data.haarcascades)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/cv2/data/


In [3]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray,1.3,5)
        #scaling factor = 1.3
        # Minimum neighbor = 5

        if faces is ():
            return None 
        for (x,y,w,h) in faces:
            cropped_face=img[y:y+h, x:x+w]
        return cropped_face
        
    cap = cv2.VideoCapture(0)
    id=3
    img_id=0

    while True:
        ret,frame = cap.read()
        if face_cropped(frame) is not None:
            img_id+=1
            face = cv2.resize(face_cropped(frame),(200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            file_name_path = "data/user."+str(id)+"."+str(img_id)+".jpg"
            cv2.imwrite(file_name_path, face)
            cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
            #(50,50) is the origin point from where text is to be written
            #font scale = 1
            #thickness = 2

            cv2.imshow("Cropped face", face)
            if cv2.waitKey(1)==13 or int(img_id)==200:
                break
    cap.release()
    cv2.destroyAllWindows()
    print("Collecting sample is completed......")
#generate_dataset()

<>:9: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
/var/folders/2p/fd9m4yqx7694_c83cy9cj_180000gn/T/ipykernel_3896/2041423167.py:9: SyntaxWarning: "is" with 'tuple' literal. Did you mean "=="?
  if faces is ():


# Train the classifier and save it 

In [4]:
import numpy as np
from PIL import Image
import os
import cv2

In [5]:
def train_classifier(data_dir):
    image_extensions = ('.jpg', '.jpeg', '.png')
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert('L');
        imageNp = np.array(img, 'uint8')
        id = int(os.path.split(image)[1].split(".")[1])

        faces.append(imageNp)
        ids.append(id)
    ids = np.array(ids)

    #Train the classifier and save 
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifier.xml")
train_classifier("data")



# Detect the face and named it if it is already stored in our dataset 

In [6]:
import cv2
import numpy as np
from PIL import Image 
import os 

In [ ]:
def draw_boundary(img,classifier,scaleFactor,minNeighbours,color,text,clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image,scaleFactor,minNeighbours)

    coords = []

    for (x,y,w,h) in features:
        cv2.rectangle(img,(x,y),(x+w,y+h),color,2)
        id,pred = clf.predict(gray_image[y:y+h,x:x+w])
        confidence = int(100*(1-pred/300))

        if confidence>75:
            if id==1:
                cv2.putText(img,"Parul",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
            if id==2:
                cv2.putText(img,"Mummy",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
            if id==3:
                cv2.putText(img,"Papa",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)

        else:
            cv2.putText(img,"Unknown",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),1,cv2.LINE_AA)


        coords=[x,y,w,h]
    return coords

def recognize(img,clf,faceCascade):
    coords = draw_boundary(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
    return img

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.xml")

video_capture = cv2.VideoCapture(0)

while True:
    ret,img = video_capture.read()
    img = recognize(img,clf,faceCascade)
    cv2.imshow("face detection", img)

    if cv2.waitKey(1)==13:
        break

video_capture.release()
cv2.destroyAllWindows()

# crop face, convert it to gray image -----> classifier
# to draw rectangle, i have to give my real image that comes from my webcam

2025-08-01 20:34:33.358 Python[3896:162103] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.
